<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/DRLforMultipleStocksTradingUsingFinRL/blob/main/FractionsTest3/3_19_fraksi4_TugasAkhirS1_Shinta_DRLforMultipleStocksTradingUsingFinRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook ini dijalankan menggunakan email lestarim1972@gmail.com

# Deep Reinforcement Learning untuk Jual-Beli Saham (Dari Awal): Jual-beli Banyak Saham 

* **Pytorch Version** 



# Daftar Isi:

* [1. Pendeskrisian Tugas](#0)
* [2. Instalasi Paket Python](#1)
    * [2.1. Instalasi Paket](#1.1)    
    * [2.2. Daftar Paket Python](#1.2)
    * [2.3. Impot Paket](#1.3)
    * [2.4. Membuat Folder](#1.4)
* [3. Mengunduh Data](#2)
* [4. Memproses Data](#3)        
    * [4.1. IndiKator Teknikal](#3.1)
    * [4.2. Melakukan *Feature Engineering*](#3.2)
* [5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style](#4)  
    * [5.1. Pemisahan Data](#4.1)  
    * [5.2. Lingkungan untuk Pelatihan *(Training)*](#4.2)    
* [6. Melatih Agen DRL](#5)
    * [6.1. Agen 1: A2C](#5.1)
    * [6.2. Agen 2: PPO](#5.2)
    * [6.3. Agen 3: TD3](#5.3)
    * [6.4. Agen 4: SAC](#5.4)
    * [6.5. Agen 5: DDPG](#5.5)
* [7. Melakukan Jual-Beli](#6)
    * [7.1. Performa di Dalam Sampel](#6.1)
    * [7.2. Performa di Luar Sampel](#6.2)
    * [7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi](#6.3)
      * [7.3.1. Agen 1: A2C](#6.3.1)
      * [7.3.2. Agen 2: PPO](#6.3.2)
      * [7.3.3. Agen 3: TD3](#6.3.3)
      * [7.3.4. Agen 4: SAC](#6.3.4)
      * [7.3.5. Agen 5: DDPG](#6.3.5)
* [8. Performa Backtesting](#7)  
    * [8.1. Status BackTesting](#7.1)
    * [8.2. Gambaran BackTesting](#7.2)

<a id='0'></a>
# Bagian 1. Pendeskrisian Tugas

Agen DRL dilatih untuk melakukan jual-beli saham. Tugas tersebut dimodelkan sebagai Markov Decision Process (MDP), dengan fungsi dan tujuannya adalah memaksimalkan pengembalian pengembalian kumulatif *(cummulative return)* yang diharapkan.

Definisi state-action-reward pada algoritam DRL kasus kali ini adalah sebagai berikut:

* **State s**: Kondisi atau *state* mewakili persepsi agen tentang lingkungan pasar. Sama seperti *trader* manusia yang menganalisis berbagai informasi, agen juga secara pasif mengamati banyak fitur dan belajar dengan cara berinteraksi dengan lingkungan pasar (biasanya dengan menjalankan ulang data historis).

* **Tindakan atau *action* a**: Ruang aksi mencakup aksi atau tindakan yang dapat dilakukan agen di setiap status atau kondisi. Misalnya, a {−1, 0, 1}, -1 berarti menjual, 0 berarti menahan, dan 1 berarti membeli. Ketika suatu aksi mengoperasikan beberapa saham, maka, a {−k, ..., 1, 0, 1, ..., k}. Misalnya, "Beli
10 saham TLKM" atau "Jual 10 saham TLKM" maka masing-masing nilai a adalah 10 atau -10.

* **Fungsi *reward* atau imbalan r(s, a, s′)**: *Reward* adalah insentif bagi agen untuk mempelajari kebijakan yang lebih baik. Misalnya *reward* dapat berupa perubahan nilai portofolio saat mengambil a pada keadaan s dan tiba pada keadaan baru s', yaitu, r(s, a, s′) = v′-v, v′ mewakili nilai portofolio pada keadaan s′ dan v mewakili nilai portofolio pada keadaan s.

* **Environment atau lingkungan jual-beli**: saham penyusun indeks JII dengan tanggal sesuai periode pengujian yang diatur.


Data untuk studi kasus ini diperoleh dari Yahoo Finance API. Data berisi harga *Open-High-Low-Close* dan *Volume*.

<a id='1'></a>
# Bagian 2. Instalasi Paket Python

<a id='1.1'></a>
## 2.1. Instalasi Paket


In [1]:
# install finrl library
!pip install git+https://github.com/shintaroudlotulhanafia/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shintaroudlotulhanafia/FinRL.git to /tmp/pip-req-build-zb8pj8h_
  Running command git clone -q https://github.com/shintaroudlotulhanafia/FinRL.git /tmp/pip-req-build-zb8pj8h_
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-7rpmuyc9/pyfolio_d7e5105b92274c3cba85543a087b55e2
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-7rpmuyc9/pyfolio_d7e5105b92274c3cba85543a087b55e2
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-7rpmuyc9/elegantrl_7e62101c7df546b897d4be58363db696
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-7rpmuyc9/elegantrl_7e62101c7df546b897d4be58363db696
     |████████████████████████████████| 2.3 MB 47.9 MB/s 
     |████████████████████████████████| 234 kB 71.2 MB/s 
     |█████████████████████

<a id='1.2'></a>
## 2.2. Daftar Paket Python
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Impot Paket

* Modul Python **pandas** digunakan untuk menganalisis dan memanipulasi data.
* Modul Python **numpy** digunakan untuk memproses larik atau array.
* Modul Python **matplotlib** digunakan membuat visualisasi data dalam dua dimensi.
* Modul Python **matplotlib.pyplot** adalah kumpulan fungsi yang membuat matplotlib berfungsi seperti MATLAB.
* Modul Python **Datetime** menyediakan sejumlah fungsi untuk menangani tanggal, waktu, dan interval waktu. Date dan datetime adalah objek dalam Python, bukan string atau timestamps.
* **YahooDownloader** menyediakan metode untuk mengambil data saham harian dari API Keuangan Yahoo!
* **FeatureEngineer** menyediakan metode untuk preprocessing data harga saham
* **data_split** membagi dataset menjadi data pelatihan dan data pengujian berdasarkan tanggal
* **StockTradingEnv** Lingkungan perdagangan saham untuk OpenAI gym
* **DRLAgent** menyediakan implementasi untuk algoritma DRL
* **DataProcessor** memproses data menggunakan prosesor data terpadu
* **backtest_stats** menghitung statistik *backtesting*
* **backtest_plot** membuat dan menampilkan plot ringkasan laporan *backtesting*.
* **get_daily_return** 
* **get_baseline** mengunduh data berdasarkan suatu indeks pada periode waktu tertentu.
* Metode **sys.path.append()** digunakan untuk menambahkan jalur sementara. Dengan demikian, jalur tersebut akan valid untuk sebuah sesi, misalnya.
* Python **itertool** adalah modul yang menyediakan berbagai fungsi yang bekerja pada iterator untuk menghasilkan iterator yang kompleks. Modul ini berfungsi sebagai alat yang cepat dan hemat memori.

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


<a id='1.4'></a>
## 2.4. Membuat Folder

In [3]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Mengunduh Data

Yahoo Finance menyediakan data saham, berita keuangan, laporan keuangan, dll, secara gratis.
* FinRL *Library* menggunakan kelas **YahooDownloader** di FinRL-Meta untuk mengambil data melalui Yahoo Finance API
* Batas Pemanggilan: Menggunakan API Publik (tanpa autentikasi), Pengguna dibatasi hingga 2.000 permintaan per jam per IP (atau hingga total 48.000 permintaan per hari).

In [4]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2017-12-31'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2019-12-20'

In [5]:
#Data diperoleh dari idx.co.id diakses pada tanggal 27/08/2022
#Fraksi 1: x<Rp200,00
fraksi1 = ['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

#Fraksi 2: Rp200,00<= x <Rp500,00
fraksi2 = ['BMTR.JK','BTON.JK','FORU.JK','GEMA.JK']

#Fraksi 3: Rp500,00<= x <Rp2.000,00
fraksi3 = ['AKRA.JK','BRPT.JK','KLBF.JK','MEDC.JK']

#Fraksi 4: Rp2000,00<= x <Rp5000,00
fraksi4 = ['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

#Fraksi 5: >=Rp5000,00
fraksi5 =['INCO.JK','INDF.JK','INTP.JK','UNTR.JK']

In [61]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = fraksi4).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10916, 8)


In [62]:
df.nunique()

date      2729
open      2064
high      2070
low       2104
close     3388
volume    5390
tic          4
day          5
dtype: int64

In [63]:
print(df['tic'].unique())

['JECC.JK' 'TMAS.JK' 'TPIA.JK' 'UNVR.JK']


* **sort_values()** berfungsi mengurutkan bingkai data dalam urutan Ascending atau Descending dari kolom yang dilewati.
* **ignore_index** berfungsi untuk menentukan untuk mereset indeks mulai dari nol. Secara default disetel *false*.
* **head()** digunakan untuk mendapatkan n baris pertama.

In [64]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,295.000000,295.000000,295.000000,198.915344,3,JECC.JK,0
1,2009-01-05,46.000000,47.000000,44.000000,34.214245,775000,TMAS.JK,0
2,2009-01-05,82.948853,82.948853,82.948853,66.105293,0,TPIA.JK,0
3,2009-01-05,1620.000000,1620.000000,1560.000000,1149.686401,20,UNVR.JK,0
4,2009-01-06,295.000000,295.000000,295.000000,198.915344,0,JECC.JK,1


<a id='3'></a>
# Bagian 4: Memproses Data

Periksa data yang hilang dan melakukan *feature engineering* untuk mengubah data menjadi *state*.
* **Menambahkan indikator teknis**

  Dalam praktik jual-beli, berbagai informasi perlu diperhitungkan, seperti harga historis, kepemilikan saham saat ini, indikator teknis, dll. Indikator teknis yang dapat digunakan antara lain, MACD, RSI, CCI, ADX, Bollinger Bands, dll. 
* **Menambahkan indeks turbulensi**

  Risk-aversion mencerminkan seberapa berani investor melakukan jual-beli saham dengan risiko yang tinggi. Hal ini mempengaruhi strategi trading investor ketika menghadapi tingkat volatilitas pasar yang berbeda. Untuk mengendalikan risiko dalam skenario terburuk, seperti krisis keuangan tahun 1998, FinRL menggunakan indeks turbulensi yang mengukur fluktuasi harga aset yang ekstrem.

In [65]:
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30"
]

Membersihkan data mentah dari nilai-nilai yang hilang. Sehingga saham dengan data yang hilang dihapus dari daftar.

In [66]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2759, 8)
Successfully added vix
Successfully added turbulence index


In [67]:
processed.nunique()

date           2652
open           2044
high           2054
low            2085
close          3334
volume         5270
tic               4
day               5
macd          10592
rsi_30         7216
cci_30        10168
vix            1441
turbulence     2398
dtype: int64

In [68]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

In [69]:
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
list_date

['2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-

In [70]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-05', 'JECC.JK'),
 ('2009-01-05', 'TMAS.JK'),
 ('2009-01-05', 'TPIA.JK'),
 ('2009-01-05', 'UNVR.JK'),
 ('2009-01-06', 'JECC.JK'),
 ('2009-01-06', 'TMAS.JK'),
 ('2009-01-06', 'TPIA.JK'),
 ('2009-01-06', 'UNVR.JK'),
 ('2009-01-07', 'JECC.JK'),
 ('2009-01-07', 'TMAS.JK'),
 ('2009-01-07', 'TPIA.JK'),
 ('2009-01-07', 'UNVR.JK'),
 ('2009-01-08', 'JECC.JK'),
 ('2009-01-08', 'TMAS.JK'),
 ('2009-01-08', 'TPIA.JK'),
 ('2009-01-08', 'UNVR.JK'),
 ('2009-01-09', 'JECC.JK'),
 ('2009-01-09', 'TMAS.JK'),
 ('2009-01-09', 'TPIA.JK'),
 ('2009-01-09', 'UNVR.JK'),
 ('2009-01-10', 'JECC.JK'),
 ('2009-01-10', 'TMAS.JK'),
 ('2009-01-10', 'TPIA.JK'),
 ('2009-01-10', 'UNVR.JK'),
 ('2009-01-11', 'JECC.JK'),
 ('2009-01-11', 'TMAS.JK'),
 ('2009-01-11', 'TPIA.JK'),
 ('2009-01-11', 'UNVR.JK'),
 ('2009-01-12', 'JECC.JK'),
 ('2009-01-12', 'TMAS.JK'),
 ('2009-01-12', 'TPIA.JK'),
 ('2009-01-12', 'UNVR.JK'),
 ('2009-01-13', 'JECC.JK'),
 ('2009-01-13', 'TMAS.JK'),
 ('2009-01-13', 'TPIA.JK'),
 ('2009-01-13', 'UNV

In [71]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915344,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105293,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915344,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,8180.000000,8225.000000,8180.000000,7612.015625,10837000.0,1.0,-70.835541,41.956731,-164.670372,12.290000,0.019867
15996,2019-12-18,JECC.JK,6175.000000,6175.000000,6175.000000,6175.000000,0.0,2.0,-2.741289,53.058014,-164.670372,12.580000,0.112172
15997,2019-12-18,TMAS.JK,98.000000,100.000000,98.000000,80.050911,2135700.0,2.0,-3.212770,38.307116,-74.912980,12.580000,0.112172
15998,2019-12-18,TPIA.JK,2487.500000,2500.000000,2481.250000,2471.146240,22134800.0,2.0,45.007680,64.526575,113.802456,12.580000,0.112172


In [72]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915344,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105293,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915344,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,8180.000000,8225.000000,8180.000000,7612.015625,10837000.0,1.0,-70.835541,41.956731,-164.670372,12.290000,0.019867
15996,2019-12-18,JECC.JK,6175.000000,6175.000000,6175.000000,6175.000000,0.0,2.0,-2.741289,53.058014,-164.670372,12.580000,0.112172
15997,2019-12-18,TMAS.JK,98.000000,100.000000,98.000000,80.050911,2135700.0,2.0,-3.212770,38.307116,-74.912980,12.580000,0.112172
15998,2019-12-18,TPIA.JK,2487.500000,2500.000000,2481.250000,2471.146240,22134800.0,2.0,45.007680,64.526575,113.802456,12.580000,0.112172


In [73]:
processed_full = processed_full.sort_values(['date','tic'])
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915344,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105293,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915344,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,8180.000000,8225.000000,8180.000000,7612.015625,10837000.0,1.0,-70.835541,41.956731,-164.670372,12.290000,0.019867
15996,2019-12-18,JECC.JK,6175.000000,6175.000000,6175.000000,6175.000000,0.0,2.0,-2.741289,53.058014,-164.670372,12.580000,0.112172
15997,2019-12-18,TMAS.JK,98.000000,100.000000,98.000000,80.050911,2135700.0,2.0,-3.212770,38.307116,-74.912980,12.580000,0.112172
15998,2019-12-18,TPIA.JK,2487.500000,2500.000000,2481.250000,2471.146240,22134800.0,2.0,45.007680,64.526575,113.802456,12.580000,0.112172


In [74]:
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915344,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105293,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915344,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,8180.000000,8225.000000,8180.000000,7612.015625,10837000.0,1.0,-70.835541,41.956731,-164.670372,12.290000,0.019867
15996,2019-12-18,JECC.JK,6175.000000,6175.000000,6175.000000,6175.000000,0.0,2.0,-2.741289,53.058014,-164.670372,12.580000,0.112172
15997,2019-12-18,TMAS.JK,98.000000,100.000000,98.000000,80.050911,2135700.0,2.0,-3.212770,38.307116,-74.912980,12.580000,0.112172
15998,2019-12-18,TPIA.JK,2487.500000,2500.000000,2481.250000,2471.146240,22134800.0,2.0,45.007680,64.526575,113.802456,12.580000,0.112172


In [75]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,295.000000,295.000000,295.000000,198.915344,3.0,0.0,0.000000,0.0,-66.666667,39.080002,0.0
1,2009-01-05,TMAS.JK,46.000000,47.000000,44.000000,34.214245,775000.0,0.0,0.000000,0.0,-66.666667,39.080002,0.0
2,2009-01-05,TPIA.JK,82.948853,82.948853,82.948853,66.105293,0.0,0.0,0.000000,0.0,-66.666667,39.080002,0.0
3,2009-01-05,UNVR.JK,1620.000000,1620.000000,1560.000000,1149.686401,20.0,0.0,0.000000,0.0,-66.666667,39.080002,0.0
4,2009-01-06,JECC.JK,295.000000,295.000000,295.000000,198.915344,0.0,1.0,0.000000,0.0,-66.666667,38.560001,0.0
5,2009-01-06,TMAS.JK,46.000000,47.000000,44.000000,33.470455,1107500.0,1.0,-0.016688,0.0,-66.666667,38.560001,0.0
6,2009-01-06,TPIA.JK,82.948853,82.948853,82.948853,66.105293,0.0,1.0,0.000000,0.0,-66.666667,38.560001,0.0
7,2009-01-06,UNVR.JK,1620.000000,1670.000000,1600.000000,1163.880005,16432500.0,1.0,0.318446,100.0,66.666667,38.560001,0.0
8,2009-01-07,JECC.JK,270.000000,270.000000,270.000000,182.058105,10000.0,2.0,-0.522742,0.0,-100.000000,43.389999,0.0
9,2009-01-07,TMAS.JK,44.000000,48.000000,44.000000,32.726669,1765000.0,2.0,-0.044328,0.0,-18.789015,43.389999,0.0


In [76]:
processed_full.nunique()

date           2652
tic               4
open           2044
high           2054
low            2085
close          3334
volume         5270
day               5
macd          10592
rsi_30         7216
cci_30        10168
vix            1441
turbulence     2398
dtype: int64

In [77]:
#memperbarui dataframe dengan bentuk harga per lot

processed_full_lot = processed_full[['open','high','low','close']].mul(100)

#drop some columns
processed_full_noOHLC = processed_full.drop(['open','high','low','close'], axis=1)
  
processed_full_lot = pd.concat([processed_full_noOHLC, processed_full_lot], axis=1)

processed_full_lot = processed_full_lot[['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'vix', 'turbulence']]

processed_full_lot

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,JECC.JK,29500.000000,29500.000000,29500.000000,19891.534424,3.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,TMAS.JK,4600.000000,4700.000000,4400.000000,3421.424484,775000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,TPIA.JK,8294.885254,8294.885254,8294.885254,6610.529327,0.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,UNVR.JK,162000.000000,162000.000000,156000.000000,114968.640137,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,JECC.JK,29500.000000,29500.000000,29500.000000,19891.534424,0.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,UNVR.JK,818000.000000,822500.000000,818000.000000,761201.562500,10837000.0,1.0,-70.835541,41.956731,-164.670372,12.290000,0.019867
15996,2019-12-18,JECC.JK,617500.000000,617500.000000,617500.000000,617500.000000,0.0,2.0,-2.741289,53.058014,-164.670372,12.580000,0.112172
15997,2019-12-18,TMAS.JK,9800.000000,10000.000000,9800.000000,8005.091095,2135700.0,2.0,-3.212770,38.307116,-74.912980,12.580000,0.112172
15998,2019-12-18,TPIA.JK,248750.000000,250000.000000,248125.000000,247114.624023,22134800.0,2.0,45.007680,64.526575,113.802456,12.580000,0.112172


<a id='4'></a>
# Bagian 5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style
Proses *training* meliputi mengamati perubahan harga saham, mengambil tindakan dan perhitungan *reward*. Dengan berinteraksi dengan lingkungan pasar, agen pada akhirnya akan memperoleh strategi perdagangan yang dapat memaksimalkan imbalan.

*Environment* pasar dibangun  menggunakan OpenAI Gym yang mensimulasikan pasar saham dengan data historis pasar.

<a id='4.1'></a>
## 5.1. Memisahkan Data
Data dibagi menjadi dua bagian, yaitu bagian pelatihan dan bagian pengujian dengan keterangan sebagai berikut:

Periode data pelatihan: 2009-01-01 hingga 2017-12-31

Periode data perdagangan: 2018-01-01 hingga 2022-08-15

In [78]:
train = data_split(processed_full_lot, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full_lot, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

8628
1980


In [79]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2155,2017-12-28,UNVR.JK,1083000.0,1088000.0,1061000.0,962058.984375,14564500.0,3.0,222.473293,67.975698,120.659953,10.18,5.395502
2156,2017-12-29,JECC.JK,470000.0,470000.0,470000.0,434712.109375,100.0,4.0,-45.023809,43.469840,-56.271744,11.04,15.259293
2156,2017-12-29,TMAS.JK,22800.0,22900.0,22800.0,18516.827393,3500.0,4.0,-1.871570,48.538841,-32.664594,11.04,15.259293
2156,2017-12-29,TPIA.JK,143125.0,150000.0,141875.0,146239.038086,24246800.0,4.0,12.320753,63.029743,167.268879,11.04,15.259293
2156,2017-12-29,UNVR.JK,1088000.0,1119500.0,1087500.0,988586.425781,12763000.0,4.0,243.419072,72.053370,162.612863,11.04,15.259293


In [80]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,JECC.JK,470000.0,470000.0,470000.0,434712.109375,0.0,1.0,-41.005663,43.469840,-55.959358,9.77,6.604093
0,2018-01-02,TMAS.JK,22900.0,22900.0,21800.0,17627.371216,16500.0,1.0,-2.067104,43.270804,-110.403857,9.77,6.604093
0,2018-01-02,TPIA.JK,150625.0,150625.0,143750.0,144411.059570,22387200.0,1.0,19.810042,60.719817,142.844633,9.77,6.604093
0,2018-01-02,UNVR.JK,1123500.0,1162000.0,1100000.0,988144.238281,12108500.0,1.0,264.457295,71.890081,158.326585,9.77,6.604093
1,2018-01-03,JECC.JK,470000.0,470000.0,470000.0,434712.109375,0.0,2.0,-38.939904,43.469840,-55.648295,9.15,24.801772


In [81]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 21


In [82]:
buy_cost_list = [0.0023] * stock_dimension
sell_cost_list = [0.0054] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

<a id='4.2'></a>
## 5.2. Lingkungan untuk Pelatihan *(Training)*



In [83]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Bagian 6: Melatih Agen DRL
* Algoritma DRL dibangun dengan menggunakan **Stable Baselines 3**. 

* FinRL mencakup algoritma DRL standar yang disempurnakan, seperti DQN, DDPG, DDPG Multi-Agen, PPO, SAC, A2C, dan TD3. FinRL juga mengizinkan pengguna untuk merancang algoritma DRL sendiri dengan mengadaptasi algoritma DRL yang telah disebutkan.

In [84]:
agent = DRLAgent(env = env_train)

**Agent Training: 5 algorithms (A2C, PPO, TD3, SAC, DDPG)**

<a id='5.1'></a>
##6.1. Agen 1: A2C


In [85]:
A2C_PARAMS = {
    "n_steps": 150, 
    "ent_coef": 0.0001, 
    "learning_rate": 0.0014,
}
model_a2c = agent.get_model("a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 150, 'ent_coef': 0.0001, 'learning_rate': 0.0014}
Using cuda device


In [86]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

--------------------------------------
| time/                 |            |
|    fps                | 369        |
|    iterations         | 100        |
|    time_elapsed       | 40         |
|    total_timesteps    | 15000      |
| train/                |            |
|    entropy_loss       | -5.61      |
|    explained_variance | -2.94e-05  |
|    learning_rate      | 0.0014     |
|    n_updates          | 99         |
|    policy_loss        | 2.49e+04   |
|    reward             | -210.03824 |
|    std                | 0.983      |
|    value_loss         | 4e+07      |
--------------------------------------
day: 2156, episode: 10
begin_total_asset: 100000000.00
end_total_asset: 707853739.57
total_reward: 607853739.57
total_cost: 1537253.70
total_trades: 7005
Sharpe: 0.775
--------------------------------------
| time/                 |            |
|    fps                | 365        |
|    iterations         | 200        |
|    time_elapsed       | 82         |
|    total_ti

<a id='5.2'></a>
##6.2. Agen 2: PPO

In [87]:
PPO_PARAMS = {
    "n_steps": 50000,
    "ent_coef": 0.7,
    "learning_rate": 0.0016,
    "batch_size": 1750,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 50000, 'ent_coef': 0.7, 'learning_rate': 0.0016, 'batch_size': 1750}
Using cuda device


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:147: UserWarning: You have specified a mini-batch size of 1750, but because the `RolloutBuffer` is of size `n_steps * n_envs = 50000`, after every 28 untruncated mini-batches, there will be a truncated mini-batch of size 1000
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=50000 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [88]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

day: 2156, episode: 30
begin_total_asset: 100000000.00
end_total_asset: 32392152.80
total_reward: -67607847.20
total_cost: 166887242.60
total_trades: 7758
Sharpe: -0.313
day: 2156, episode: 40
begin_total_asset: 100000000.00
end_total_asset: 202578567.29
total_reward: 102578567.29
total_cost: 241935561.32
total_trades: 8014
Sharpe: 0.457
---------------------------------
| time/              |          |
|    fps             | 369      |
|    iterations      | 1        |
|    time_elapsed    | 135      |
|    total_timesteps | 50000    |
| train/             |          |
|    reward          | 66.03007 |
---------------------------------


<a id='5.3'></a>
##6.3. Agen 3: TD3

In [89]:
TD3_PARAMS = {
    "batch_size": 7000, 
    "buffer_size": 10000, 
    "learning_rate": 0.9  
}
model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 7000, 'buffer_size': 10000, 'learning_rate': 0.9}
Using cuda device


In [90]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000)

day: 2156, episode: 50
begin_total_asset: 100000000.00
end_total_asset: 828202554.63
total_reward: 728202554.63
total_cost: 2323292.97
total_trades: 6087
Sharpe: 0.954
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 84        |
|    time_elapsed    | 101       |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 2.03e+06  |
|    critic_loss     | 4.07e+10  |
|    learning_rate   | 0.9       |
|    n_updates       | 6471      |
|    reward          | 273.31757 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 74        |
|    time_elapsed    | 232       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 1.67e+06  |
|    critic_loss     | 5.04e+09  |
|    learning_rate   | 0.9       |
|    n_updates       | 1509

<a id='5.4'></a>
##6.4. Agen 4: SAC

In [112]:
SAC_PARAMS = {
    "ent_coef": 1.25,
    "learning_rate": 0.0001,
    "batch_size": 4000,
    "buffer_size": 5000,
    "learning_starts": 14,
    
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'ent_coef': 1.25, 'learning_rate': 0.0001, 'batch_size': 4000, 'buffer_size': 5000, 'learning_starts': 14}
Using cuda device


In [113]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

day: 2156, episode: 120
begin_total_asset: 100000000.00
end_total_asset: 1272898189.45
total_reward: 1172898189.45
total_cost: 229472.08
total_trades: 4312
Sharpe: 0.923
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 62       |
|    time_elapsed    | 137      |
|    total_timesteps | 8628     |
| train/             |          |
|    actor_loss      | 1.42e+04 |
|    critic_loss     | 9.81e+05 |
|    ent_coef        | 1.25     |
|    learning_rate   | 0.0001   |
|    n_updates       | 8613     |
|    reward          | 5651.416 |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 62       |
|    time_elapsed    | 276      |
|    total_timesteps | 17256    |
| train/             |          |
|    actor_loss      | 8.03e+03 |
|    critic_loss     | 1.23e+06 |
|    ent_coef        | 1.25     |
|    learning_

<a id='5.5'></a>
##6.5. Agen 5: DDPG

In [93]:
DDPG_PARAMS = {
    "batch_size": 1250, 
    "buffer_size": 50000, 
    "learning_rate": 0.0003
}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 1250, 'buffer_size': 50000, 'learning_rate': 0.0003}
Using cuda device


In [94]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=60000)

day: 2156, episode: 90
begin_total_asset: 100000000.00
end_total_asset: 1077151048.22
total_reward: 977151048.22
total_cost: 229464.09
total_trades: 4312
Sharpe: 0.777
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 122       |
|    time_elapsed    | 70        |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 790       |
|    critic_loss     | 1.6e+07   |
|    learning_rate   | 0.0003    |
|    n_updates       | 6471      |
|    reward          | 1390.7354 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 107       |
|    time_elapsed    | 160       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | -457      |
|    critic_loss     | 2.69e+06  |
|    learning_rate   | 0.0003    |
|    n_updates       | 1509

<a id='6'></a>
# Bagian 7: Melakukan Jual-Beli

<a id='6.1'></a>
## 7.1. Performa di Dalam Sampel

Asumsikan modal awal adalah Rp100.000.000.

In [95]:
data_risk_indicator = processed_full_lot[(processed_full_lot.date<TRAIN_END_DATE) & (processed_full_lot.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [96]:
insample_risk_indicator.vix.describe()

count    2157.000000
mean       18.666583
std         7.813415
min         9.140000
25%        13.370000
50%        16.299999
75%        21.540001
max        56.650002
Name: vix, dtype: float64

In [97]:
insample_risk_indicator.vix.quantile(0.996)

48.84047967529307

In [98]:
insample_risk_indicator.turbulence.describe()

count    2157.000000
mean        3.855362
std        11.346045
min         0.000000
25%         0.269254
50%         1.242756
75%         3.754829
max       306.793339
Name: turbulence, dtype: float64

In [99]:
insample_risk_indicator.turbulence.quantile(0.996)

50.30349656735339

<a id='6.2'></a>
##7.2. Performa di Luar Sampel

FinRL dapat diatur dengan cara melakukan pelatihan ulang secara berkala, misalnya, pelatihan ulang setiap tiga bulan, bulanan, atau mingguan. 

Pada notebook ini, *hyperparameter* hanya diatur satu kali dengan menggunakan data sampel selama periode yang telah ditetapkan. Sehingga, terjadi beberapa peluruhan alfa seiring dengan perpanjangan tanggal perdagangan.

Banyak *hyperparameter* – mis. learning rate, mempengaruhi proses pembelajaran dan biasanya ditentukan dengan menguji beberapa variasi.

In [100]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

<a id='6.3'></a>
##7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi

<a id='6.3.1'></a>
###7.3.1. Agen 1: A2C

In [101]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym)

hit end!


<a id='6.3.2'></a>
###7.3.2. Agen 2: PPO

In [102]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym)

hit end!


<a id='6.3.3'></a>
###7.3.3. Agen 3: TD3

In [103]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym)

hit end!


<a id='6.3.4'></a>
###7.3.4. Agen 4: SAC

In [114]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


<a id='6.3.5'></a>
###7.3.5. Agen 5: DDPG

In [105]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

hit end!


<a id='7'></a>
# Bagian 8: Performa Backtesting
Backtesting memiliki peran kunci dalam mengevaluasi kinerja strategi perdagangan. Alat backtesting otomatis lebih disukai karena dapat meminimalisir kesalahan manusia. Backtesting dapat dilakukan dengan menggunakan paket Quantopian pyfolio untuk menguji strategi perdagangan pada notebook ini. Backtesting tersebut mudah digunakan dan terdiri dari berbagai plot yang memberikan gambaran komprehensif tentang kinerja strategi perdagangan.

<a id='7.1'></a>
## 8.1 Status BackTesting


###8.1.1 Agen A2C

In [106]:
print("===========Get Backtest Results of A2C Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_a2c)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_a2c_"+now+'.csv')

===========Get Backtest Results of A2C Model===========
Annual return         -0.053332
Cumulative returns    -0.102063
Annual volatility      0.277127
Sharpe ratio          -0.060809
Calmar ratio          -0.187166
Stability              0.111085
Max drawdown          -0.284943
Omega ratio            0.988448
Sortino ratio         -0.090718
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.047802
Daily value at risk   -0.034982
dtype: float64


###8.1.2 Agen ppo

In [107]:
print("===========Get Backtest Results of PPO Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ppo_"+now+'.csv')

===========Get Backtest Results of PPO Model===========
Annual return         -0.106547
Cumulative returns    -0.198524
Annual volatility      0.233500
Sharpe ratio          -0.366935
Calmar ratio          -0.403140
Stability              0.150237
Max drawdown          -0.264293
Omega ratio            0.935660
Sortino ratio         -0.519189
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.999184
Daily value at risk   -0.029758
dtype: float64


###8.1.3 Agen TD3

In [108]:
print("===========Get Backtest Results of TD3 Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_td3)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_td3_"+now+'.csv')

===========Get Backtest Results of TD3 Model===========
Annual return         -0.121121
Cumulative returns    -0.224002
Annual volatility      0.251236
Sharpe ratio          -0.389510
Calmar ratio          -0.421025
Stability              0.129992
Max drawdown          -0.287681
Omega ratio            0.933022
Sortino ratio         -0.549422
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.003659
Daily value at risk   -0.032041
dtype: float64


###8.1.4 Agen SAC

In [115]:
print("===========Get Backtest Results of SAC Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_sac)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_sac_"+now+'.csv')

===========Get Backtest Results of SAC Model===========
Annual return          0.276368
Cumulative returns     0.614979
Annual volatility      0.330771
Sharpe ratio           0.902559
Calmar ratio           0.855195
Stability              0.223496
Max drawdown          -0.323164
Omega ratio            1.188640
Sortino ratio          1.413861
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.121310
Daily value at risk   -0.040489
dtype: float64


###8.1.5 Agen DDPG

In [110]:
print("===========Get Backtest Results of DDPG Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ddpg)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ddpg_"+now+'.csv')

===========Get Backtest Results of DDPG Model===========
Annual return          0.232674
Cumulative returns     0.508175
Annual volatility      0.344204
Sharpe ratio           0.778238
Calmar ratio           0.776818
Stability              0.492218
Max drawdown          -0.299522
Omega ratio            1.309705
Sortino ratio          1.299552
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.027440
Daily value at risk   -0.042303
dtype: float64


###8.1.6. Baseline Stats

In [111]:
#baseline stats
print("===========Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^JKII", 
        start = df_account_value_a2c.loc[0,'date'],
        end = df_account_value_a2c.loc[len(df_account_value_a2c)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

===========Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (478, 8)
Annual return         -0.041972
Cumulative returns    -0.078113
Annual volatility      0.185511
Sharpe ratio          -0.138819
Calmar ratio          -0.175216
Stability              0.093175
Max drawdown          -0.239543
Omega ratio            0.977818
Sortino ratio         -0.189296
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.900451
Daily value at risk   -0.023474
dtype: float64
